<a href="https://colab.research.google.com/github/pvgandhi404/Resume-Classifier/blob/main/Adanomad_Text_CLassification_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resume Classifier

The resume classifier will catagorize a resume into 3 different job catagories:

0: Software Engineer

1: Data Scientist

2: Project Manager


To run the machine learning model, simply scroll down to the "Model Testing" section and hit run on both code blocks. Should an error be caused, please instead select "run all".  

 To insert custom inputs for the ML mode, simply change the variable "text" with the desired input. The code will automatically process the data and print the corresponding job title.


Explanation:
- The project was implemented using the Transformers library from HuggingFace and TensorFlow.

- I was unable to find an appropriate dataset for project manager jobs. To ensure consistancy and prvent bias in the results, I decided to forgo using the datasets for software engineer and data scientist.

- Instead, I simply asked chatGPT to give me a list of common job descriptions for each role. I then used that list to fine-tune the model.

- Since the dataset is quite small, there is a likely chance of reduced accuracy due to both underfitting and overfitting data.

Strategies:
- To build the resume classifier, I used the "finbert" text classification model from Hugginface, mainly due to its popularity and ability to classify between 3 different labels. This allowed me to easily translate the model into classifying 3 different jobs.

- I used the natural language took kit (ntlk) to remove all stopwords from all test and validation data. The same is done for all data before being processed by the model, thus increasing accuracy.

# Imports and Installs



In [1]:
!pip install tensorflow
!pip install transformers -q
!pip install nltk

In [2]:
import nltk
from nltk.corpus import stopwords

from sklearn.metrics import accuracy_score

from google.colab import drive

import numpy as np

import tensorflow as tf

from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TFTrainer, TFTrainingArguments, Trainer
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from transformers import TFTrainer, TFTrainingArguments

In [3]:
# @title Improvised Dataset: contains train_texts and train_labels (hidden for convenince)

# 1rst 20 elements are software engineer descriptions
# Middle 20 are data science descriptions
# Last 20 are project manager descriptions
train_texts = ["Develop, test, and maintain software solutions according to specifications.",
"Troubleshoot, debug, and resolve software defects and technical issues.",
"Design and implement user-facing features using HTML, CSS, and JavaScript frameworks.",
"Collaborate with UX/UI designers to create intuitive and responsive web interfaces.",
"Optimize application performance and ensure cross-browser compatibility.",
"Write efficient, reusable, and modular code for frontend components.",
"Stay updated with emerging frontend technologies and industry trends.",
"Build scalable, high-performance backend systems and APIs using programming languages like Python, Java, or Node.js.",
"Design and implement database schemas and data models for efficient storage and retrieval.",
"Ensure system security, data protection, and compliance with industry standards.",
"Optimize backend processes for speed, scalability, and reliability.",
"Collaborate with frontend developers to integrate frontend and backend components.",
"Develop end-to-end web applications, handling both frontend and backend components.",
"Architect and design software systems to meet functional and non-functional requirements.",
"Implement features across the entire software stack, from database design to user interface development.",
"Collaborate with product managers, designers, and other stakeholders to prioritize and deliver features.",
"Conduct code reviews, testing, and debugging to ensure high-quality software delivery.",
"Automate infrastructure provisioning, deployment, and management using tools like Docker, Kubernetes, and Terraform.",
"Implement continuous integration and continuous deployment (CI/CD) pipelines to streamline software delivery.",
"Monitor system performance, reliability, and availability using monitoring tools like Prometheus and Grafana.",

"Analyze large datasets to extract actionable insights and drive business decisions.",
"Develop machine learning models and algorithms for predictive analytics and forecasting.",
"Collaborate with cross-functional teams to identify business opportunities and solve complex problems.",
"Use statistical techniques and data visualization tools to communicate findings and recommendations.",
"Continuously explore and implement new data science methodologies and technologies.",
"Perform exploratory data analysis to uncover trends, patterns, and insights in data.",
"Create dashboards, reports, and visualizations to present findings to stakeholders.",
"Develop and maintain data pipelines for data extraction, transformation, and loading (ETL).",
"Collaborate with business users to understand requirements and provide data-driven solutions.",
"Conduct statistical analysis and hypothesis testing to support decision-making processes.",
"Design, build, and maintain data infrastructure and pipelines for processing and storing large datasets.",
"Develop and optimize data workflows for data ingestion, transformation, and storage.",
"Implement data quality checks and monitoring to ensure data integrity and reliability.",
"Collaborate with data scientists and analysts to support their data needs and requirements.",
"Stay updated with emerging technologies and best practices in data engineering and big data.",
"Gather and analyze business data to identify trends, opportunities, and challenges.",
"Design and develop reports, dashboards, and interactive visualizations using BI tools.",
"Translate business requirements into technical specifications for data queries and reports.",
"Collaborate with stakeholders to define KPIs and metrics for measuring business performance.",
"Provide insights and recommendations to support strategic decision-making.",

"Plan, execute, and oversee project activities to ensure successful completion within scope, budget, and schedule.",
"Define project objectives, deliverables, and milestones in collaboration with stakeholders.",
"Allocate resources, manage budgets, and track project progress using project management tools.",
"Identify risks, issues, and dependencies and implement mitigation strategies to minimize project disruptions.",
"Facilitate communication and collaboration among project team members and stakeholders.",
"Lead IT projects from initiation to closure, including software development, infrastructure upgrades, and system implementations.",
"Coordinate with technical teams, business stakeholders, and vendors to deliver IT solutions that meet business requirements.",
"Manage project scope, budget, and timelines while ensuring adherence to quality standards and regulatory requirements.",
"Conduct project risk assessments and develop mitigation plans to address potential issues.",
"Provide regular project status updates and reports to senior management and stakeholders.",
"Lead infrastructure projects, such as network upgrades, data center migrations, and cloud migrations, from planning to implementation.",
"Develop project plans, schedules, and budgets in alignment with organizational goals and technical requirements.",
"Collaborate with cross-functional teams, including IT, operations, and facilities, to coordinate project activities and deliverables.",
"Manage vendor relationships, contracts, and service level agreements (SLAs) to ensure project success and stakeholder satisfaction.",
"Monitor project performance metrics and KPIs to track progress and identify areas for improvement.",
"Facilitate agile ceremonies, including sprint planning, daily stand-ups, sprint reviews, and retrospectives, to support agile development methodologies.",
"Manage agile project backlogs, prioritize user stories, and track sprint progress using agile project management tools.",
"Coach and mentor agile teams on agile principles, practices, and tools to improve team performance and delivery outcomes.",
"Collaborate with product owners, scrum masters, and stakeholders to define project goals, scope, and acceptance criteria.",
"Foster a culture of continuous improvement and innovation within agile teams and across the organization."
               ]

# 0 = Software engineer
# 1 = data scientist
# 2 = project manager
train_labels = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

print(len(train_texts))
print(len(train_labels))

60
60


# Remove Stopwords

In [4]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Function to remove all stop words from a string
def remove_stopwords(text):
  word_list = text.split()

  filtered_list = [word for word in word_list if word not in stop_words]
  return ' '.join(filtered_list)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
# Remove all stopwords from training data
for i in range(0, len(train_texts)):
  train_texts[i] = remove_stopwords(train_texts[i])

# Train the Model and Save

In [6]:
# Load pre-trained model and tokenizer
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

# Tokenize training data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors="tf")

# Prepare TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).shuffle(len(train_texts)).batch(8)

# Define optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Fine-tune the model
model.fit(train_dataset, epochs=3)

# Save the fine-tuned model
model.save_pretrained("/content/drive/MyDrive/Colab_Notebooks/Adanomad")
tokenizer.save_pretrained("/content/drive/MyDrive/Colab_Notebooks/Adanomad")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/438M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at ProsusAI/finbert and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
8/8 [==============================] - 83s 4s/step - loss: 1.1197 - accuracy: 0.4000
Epoch 2/3
8/8 [==============================] - 32s 4s/step - loss: 1.0205 - accuracy: 0.4500
Epoch 3/3
8/8 [==============================] - 29s 4s/step - loss: 0.8279 - accuracy: 0.7500


('/content/drive/MyDrive/Colab_Notebooks/Adanomad/tokenizer_config.json',
 '/content/drive/MyDrive/Colab_Notebooks/Adanomad/special_tokens_map.json',
 '/content/drive/MyDrive/Colab_Notebooks/Adanomad/vocab.txt',
 '/content/drive/MyDrive/Colab_Notebooks/Adanomad/added_tokens.json',
 '/content/drive/MyDrive/Colab_Notebooks/Adanomad/tokenizer.json')

# Model Evualuation

In [7]:
# Test data was also generated using chatGPT
test_data = [
    "Developed robust and scalable web applications using modern technologies like React.js and Node.js.",
    "Designed and implemented RESTful APIs to facilitate communication between frontend and backend systems.",
    "Automated deployment processes using tools like Jenkins, Docker, and Kubernetes to streamline the release cycle.",
    "Conducted exploratory data analysis to identify patterns and trends in customer behavior.",
    "Clean, preprocess, and prepare raw data for analysis and modeling.",
    "Managed project scope, schedule, and resources to ensure successful project outcomes.",
    "Developed project plans and schedules using tools like Microsoft Project and Smartsheet.",
]

predictions = []

for i in range(0, len(test_data)):
  test_text = remove_stopwords(test_data[i])

  # Tokenize the input text
  inputs = tokenizer(test_text, return_tensors="tf")

  # Perform inference
  outputs = model(**inputs)

  # Access logits
  logits = outputs.logits

  # Convert logits to probabilities
  probabilities = tf.nn.softmax(logits, axis=-1)

  # Access predicted class
  predicted_class = tf.argmax(probabilities, axis=-1).numpy()[0]

  predictions.append(predicted_class)

# True values
test_labels = [0, 0, 0, 1, 1, 2, 2]

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.8571428571428571


# Model Testing

In [8]:
# Mount Google Drive
drive.mount('/content/drive')

# Path to the saved fine-tuned model directory on your local filesystem
fine_tuned_model_path = "/content/drive/MyDrive/Colab_Notebooks/Adanomad"

# Load the fine-tuned model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)
model = TFAutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some layers from the model checkpoint at /content/drive/MyDrive/Colab_Notebooks/Adanomad were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/Colab_Notebooks/Adanomad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [9]:
# Example text for classification
text = "Collaborate with cross-functional teams to define project objectives, gather requirements, and deliver data-driven solutions.Develop and deploy machine learning models for forecasting, classification, and recommendation systems. Design and implement data pipelines for data ingestion, preprocessing, and feature engineering.."
text = remove_stopwords(text)

# Tokenize the input text
inputs = tokenizer(text, return_tensors="tf")

# Perform inference
outputs = model(**inputs)

# Access logits
logits = outputs.logits

# Convert logits to probabilities
probabilities = tf.nn.softmax(logits, axis=-1)

# Access predicted class
predicted_class = tf.argmax(probabilities, axis=-1).numpy()[0]

print("Predicted class:", predicted_class)

if(predicted_class == 0):
  print("Job: Software Engineer")
elif(predicted_class == 1):
  print("Job: Data Scientist")
elif(predicted_class == 2):
  print("Job: Project Manager")


Predicted class: 1
Job: Data Scientist
